In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers datasets torch rouge_score PyPDF2 kaggle

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ed886f00737a08d4e714e4cc9892c02a48c4e9fc832c83e57614d06296b69fd5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
#!git clone https://github.com/feralvam/easse.git
#!pip install -e easse
#!pip install sacrebleu

In [4]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import PyPDF2
from typing import Dict, List, Tuple, Union
import json

In [5]:
SEED = 224
LEARNING_RATE = 1e-5
NUM_EPOCHS = 2
BATCH_SIZE = 16
MAX_SOURCE_LENGTH = 128
MAX_TARGET_LENGTH = 64
WEIGHT_DECAY = 0.01
GRAD_ACCUMULATION_STEPS = 8

In [6]:
billsum = load_dataset('billsum')
billsum_train = billsum['train']
billsum_test = billsum['test']
billsum_ca_test = billsum['ca_test']

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [7]:
datasets = {
    'billsum': {
        'train_val': billsum_train,
        'test': billsum_test,
        'ca_test': billsum_ca_test
    }
}

In [8]:
results = {}

In [9]:
dataset_name = 'billsum'
dataset_split = datasets[dataset_name]
print(f"Training on {dataset_name} dataset..")
train_data = dataset_split['train_val']
val_data = dataset_split['test']

print(f"Training with seed: {SEED}, learning rate: {LEARNING_RATE}, epochs: {NUM_EPOCHS}")

Training on billsum dataset..
Training with seed: 224, learning rate: 1e-05, epochs: 2


In [10]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [11]:
def preprocess_batch(examples):
    if 'summary' in examples:
        inputs = examples['text']
        targets = examples['summary']
    elif 'title' in examples:
        inputs = examples['text']
        targets = examples['title']
    else:
        inputs = examples['text']
        targets = [""] * len(inputs)
        
    model_inputs = tokenizer(inputs, max_length=MAX_SOURCE_LENGTH, padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=MAX_TARGET_LENGTH, padding='max_length', truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [12]:
train_dataset = train_data.map(
    preprocess_batch,
    batched=True,
    remove_columns=train_data.column_names
)
val_dataset = val_data.map(
    preprocess_batch,
    batched=True,
    remove_columns=val_data.column_names
)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

In [14]:
training_args = TrainingArguments(
    output_dir=f"./results/{dataset_name}",
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    seed=SEED,
    report_to="none",
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [16]:
trainer.train()

Step,Training Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=296, training_loss=1.932258399757179, metrics={'train_runtime': 1466.001, 'train_samples_per_second': 25.851, 'train_steps_per_second': 0.202, 'total_flos': 1.0260427626774528e+16, 'train_loss': 1.932258399757179, 'epoch': 1.9983122362869197})

In [17]:
trainer.save_model('bart')
model.save_pretrained(f"./models/{dataset_name}")
tokenizer.save_pretrained(f"./models/{dataset_name}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

('./models/billsum/tokenizer_config.json',
 './models/billsum/special_tokens_map.json',
 './models/billsum/vocab.json',
 './models/billsum/merges.txt',
 './models/billsum/added_tokens.json')

In [18]:
eval_results = trainer.evaluate()
results[f"{dataset_name}_{SEED}_{LEARNING_RATE}_{NUM_EPOCHS}"] = eval_results

In [19]:
output_file = "model_outputs.zip"
temp_dir = "temp_outputs"
os.makedirs(temp_dir, exist_ok=True)

In [20]:
import shutil
try:
    # Save models
    if os.path.exists("./models"):
        models_temp_dir = os.path.join(temp_dir, "models")
        os.makedirs(models_temp_dir, exist_ok=True)
        for dataset_dir in os.listdir("./models"):
            dataset_path = os.path.join("./models", dataset_dir)
            if os.path.isdir(dataset_path):
                dst_path = os.path.join(models_temp_dir, dataset_dir)
                shutil.copytree(dataset_path, dst_path)
    
    # Save results
    if os.path.exists("./results"):
        results_temp_dir = os.path.join(temp_dir, "results")
        os.makedirs(results_temp_dir, exist_ok=True)
        for dataset_dir in os.listdir("./results"):
            dataset_path = os.path.join("./results", dataset_dir)
            if os.path.isdir(dataset_path):
                dst_path = os.path.join(results_temp_dir, dataset_dir)
                shutil.copytree(dataset_path, dst_path)
    
    # Save training results JSON
    if os.path.exists("training_results.json"):
        shutil.copy2("training_results.json", temp_dir)
    
    # Create zip archive
    if os.path.exists(output_file):
        os.remove(output_file)
    shutil.make_archive(output_file.replace('.zip', ''), 'zip', temp_dir)
    
finally:
    # Cleanup
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

In [21]:
print("\nGenerating summaries for test examples...")
model = BartForConditionalGeneration.from_pretrained(f"./models/{dataset_name}")
tokenizer = BartTokenizer.from_pretrained(f"./models/{dataset_name}")
model.to(device)
model.eval()


Generating summaries for test examples...


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [22]:
test_examples = dataset_split['test'].select(range(5))

In [23]:
generated_summaries = []
reference_summaries = []

In [24]:
with torch.no_grad():
    for example in test_examples:
        # Tokenize input text
        inputs = tokenizer(
            example['text'],
            max_length=MAX_SOURCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate summary
        summary_ids = model.generate(
            inputs['input_ids'],
            num_beams=4,
            max_length=MAX_TARGET_LENGTH,
            length_penalty=2.0,
            early_stopping=True,
            no_repeat_ngram_size=3
        )

        # Decode generated summary
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        generated_summaries.append(generated_summary)
        reference_summaries.append(example['summary'])

        # Print results
        print("\nOriginal Text:", example['text'][:200] + "...")
        print("\nGenerated Summary:", generated_summary)
        print("\nReference Summary:", example['summary'])
        print("\n" + "="*80)



Original Text: SECTION 1. ENVIRONMENTAL INFRASTRUCTURE.

    (a) Jackson County, Mississippi.--Section 219 of the Water 
Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is 
amended--
        (1) i...

Generated Summary: Amends the Water Resources Development Act of 1992 to direct the Secretary of the Interior to provide for the provision of an alternative water supply and a project for the construction of a new water treatment facility in Jackson County, Mississippi. 
Directs the Secretary to:  (1) establish an alternative

Reference Summary: Amends the Water Resources Development Act of 1999 to: (1) authorize appropriations for FY 1999 through 2009 for implementation of a long-term resource monitoring program with respect to the Upper Mississippi River Environmental Management Program (currently, such funding is designated for a program for the planning, construction, and evaluation of measures for fish and wildlife habitat rehabilitation and enhancement); (2) auth

In [25]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []

In [26]:
for gen, ref in zip(generated_summaries, reference_summaries):
    score = scorer.score(gen, ref)
    rouge_scores.append(score)

In [27]:
avg_rouge_scores = {
    'rouge1': np.mean([score['rouge1'].fmeasure for score in rouge_scores]),
    'rouge2': np.mean([score['rouge2'].fmeasure for score in rouge_scores]),
    'rougeL': np.mean([score['rougeL'].fmeasure for score in rouge_scores])
}

In [28]:
print("\nAverage ROUGE Scores:")
for metric, score in avg_rouge_scores.items():
    print(f"{metric}: {score:.4f}")


Average ROUGE Scores:
rouge1: 0.4087
rouge2: 0.1940
rougeL: 0.3061


In [29]:
summary_results = {
    'generated_summaries': generated_summaries,
    'reference_summaries': reference_summaries,
    'rouge_scores': avg_rouge_scores
}

In [30]:
summary_results

{'generated_summaries': ['Amends the Water Resources Development Act of 1992 to direct the Secretary of the Interior to provide for the provision of an alternative water supply and a project for the construction of a new water treatment facility in Jackson County, Mississippi. \nDirects the Secretary to:  (1) establish an alternative',
  'Federal Forage Fee Act of 1993 - Directs the Secretary of Agriculture (USDA) to: (1) establish a fee for the use of public lands for the production of livestock forage, wildlife forage and recreation and other multiple use opportunities; and (2) require the Secretary to establish',
  'Merchant Marine of World War II Congressional Gold Medal Act - Directs the Secretary of Veterans Affairs (VA) to award a gold medal to a member of the Merchant Marine of the Armed Forces of the United States of America who: (1) served in the Pacific Ocean during World War Two;',
  'Small Business Tax Modernization Act of 2004 - Amends the Internal Revenue Code to: (1) te

In [31]:
from IPython.display import FileLink
FileLink(r'model_outputs.zip')

/kaggle/working/model_outputs.zip

In [32]:
from IPython.display import FileLink
FileLink(r'models/billsum/model.safetensors')

/kaggle/working/models/billsum/model.safetensors

In [33]:
from IPython.display import FileLink
FileLink(r'results/billsum//checkpoint-296/model.safetensors')

/kaggle/working/results/billsum/checkpoint-296/model.safetensors

In [34]:
from IPython.display import FileLink
FileLink(r'results/billsum/checkpoint-296/optimizer.pt')

/kaggle/working/results/billsum/checkpoint-296/optimizer.pt